<span style="font-size: 14pt">MIPT, Applied ML, Autumn 2018</span>

<span style="font-size: 16pt"> HW #4: Approximate RL homework

<span style="color:red; font-size: 14pt;"> Дедлайн 19.11.2018 23:59 </span>

<span style="color:blue; font-size: 12pt">Valentin Malykh </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">val@maly.hk</span>

**Оформление дз**: 
- Выполненное задание требуется отправлять через <a href='https://goo.gl/forms/XPSIbwp7wPxB4SsI3'>форму </a>

- Выполненное дз прикрепляйте в формате файла ``<фамилия>_<группа>_task<номер>.ipynb``, например: ``ivanov_594_task4.ipynb`` 

**Вопросы**:
- Вопросы присылайте в канал в телеграмме ``[Fall 2018]ML Seminars``

--------
- **PS1**: Будьте внимательны при заполнении формы, когда отправляете ДЗ 

<h1 align="center">Практическое задание (100%)</h1>
Описание находится по ссылке: https://gist.github.com/madrugado/1262c3077bf7d8ac8166e4350f0f67e2

При выполнении задания использовался https://github.com/dgriff777/a3c_continuous

Модель обучается примерно 15 часов, после чего выдает результат на 300 эпохах примерно $300 \pm 10$.

Результаты промежучного обучения сохраняются в файл best_model. В файле current_best_model уже обученная модель.

In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import gym

from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm
import math


%load_ext autoreload
%autoreload 2
%matplotlib inline



In [2]:
from __future__ import division
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from collections import defaultdict

from torch.autograd import Variable

import math
import numpy as np
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torch.multiprocessing as mp

In [2]:
#эта ячейка запускает обучение модели.
import subprocess
subprocess.Popen(["python", "main.py"])

In [3]:
from main import A3C_Model

In [4]:
def evaluate(num_runs, model_file, render=False):
    env = gym.make('BipedalWalker-v2')
    action_dim = env.action_space.shape[0]
    state_dim = env.observation_space.shape[0]
    actor = A3C_Model(state_dim, action_dim)
    rewards = []

    for i in range(num_runs):
        state_dict = torch.load(model_file)
        actor.load_state_dict(state_dict)
        actor.eval()
        total_reward = 0
        new_obs = env.reset()
        actor.train(False)
        cx = Variable(torch.zeros(1, 128))
        hx = Variable(torch.zeros(1, 128))
        while True:
            if render:
                env.render()
            obs = torch.from_numpy(new_obs).float()
            value, mu, sigma, (hx, cx) = actor((Variable(obs), (hx, cx)))
            mu = torch.clamp(mu.data, -1.0, 1.0)
            action = mu.cpu().numpy()[0]
            new_obs, reward, done, info = env.step(action)
            total_reward += reward
            if done:
                rewards.append(total_reward)
                break
    print(np.mean(rewards))

In [7]:
evaluate(300, 'current_best_model')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
300.0299086789108


In [ ]:
evaluate(300, 'best_model')